## Setup

### Conda/Miniconda
[Download](https://conda.io/miniconda.html) and install Miniconda. Select the Python 3.7 version or later. Don't select the Python 2.x version.

### AzureML Python SDK
Install the Python SDK:  make sure to install notebook, and contrib
```
conda create -n azureml -y Python=3.6
conda activate azureml
pip install --upgrade azureml-sdk[notebooks,contrib] scikit-image tensorboard tensorboardX --user 
conda install ipywidgets
jupyter nbextension install --py --user azureml.widgets
jupyter nbextension enable azureml.widgets --user --py
```
You will need to restart jupyter after this
Detailed instructions are here: https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python 

### (Optional) Install VS Code and the VS Code extension 
[Download](https://code.visualstudio.com/) and install Visual Studio Code then the [Azure Machine Learning Extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.vscode-ai)
Make sure it has a recent version of the Python SDK -- remove the folder ~/.azureml/envs if there are issuse. A current SDK will be installed when you first use AML from VSCode.

### Clone this repository
```
git clone https://github.com/maxluk/dogbreeds-webinar
jupyter notebook
```

# Dog breed classification using Pytorch Estimators on Azure Machine Learning service

Have you ever seen a dog and not been able to tell the breed? Some dogs look so similar, that it can be nearly impossible to tell. For instance these are a few breeds that are difficult to tell apart:

#### Alaskan Malamutes vs Siberian Huskies
![Image of Alaskan Malamute vs Siberian Husky](http://cdn.akc.org/content/article-body-image/malamutehusky.jpg)

#### Whippet vs Italian Greyhound 
![Image of Whippet vs Italian Greyhound](http://cdn.akc.org/content/article-body-image/whippetitalian.jpg)

There are sites like http://what-dog.net, which use Microsoft Cognitive Services to be able to make this easier. 

In this tutorial, you will learn how to train a Pytorch image classification model using transfer learning with the Azure Machine Learning service. The Azure Machine Learning python SDK's [PyTorch estimator](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-pytorch) enables you to easily submit PyTorch training jobs for both single-node and distributed runs on Azure compute. The model is trained to classify dog breeds using the [Stanford Dog dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) and it is based on a pretrained ResNet18 model. This ResNet18 model has been built using images and annotation from ImageNet. The Stanford Dog dataset contains 120 classes (i.e. dog breeds), to save time however, for most of the tutorial, we will only use a subset of this dataset which includes only 10 dog breeds.

## What is Azure Machine Learning service?
Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](aml-overview.png)


## How can we use it for training image classification models?
Training machine learning models, particularly deep neural networks, is often a time- and compute-intensive task. Once you've finished writing your training script and running on a small subset of data on your local machine, you will likely want to scale up your workload.

To facilitate training, the Azure Machine Learning Python SDK provides a high-level abstraction, the estimator class, which allows users to easily train their models in the Azure ecosystem. You can create and use an Estimator object to submit any training code you want to run on remote compute, whether it's a single-node run or distributed training across a GPU cluster. For PyTorch and TensorFlow jobs, Azure Machine Learning also provides respective custom PyTorch and TensorFlow estimators to simplify using these frameworks.

### Steps to train with a Pytorch Estimator:
In this tutorial, we will:
- Connect to an Azure Machine Learning service Workspace 
- Create a remote compute target
- Upload your training data (Optional)
- Create your training script
- Create an Estimator object
- Submit your training job

## Create workspace

In the next step, you will create your own Workspace to use in this tutorial.

**You will be asked to login during this step. Please use your Microsoft AAD credentials.**

## Prerequisites
Make sure you have access to an Azure subscription. Your group's admin should have added you to your team's subscription. The subscriptions are listed [here](https://microsoft.sharepoint.com/teams/azuremlnursery/_layouts/OneNote.aspx?id=%2Fteams%2Fazuremlnursery%2FSiteAssets%2FAzure%20ML%20Nursery%20Notebook&wd=target%28Workshop.one%7C265D85D5-44C8-9D40-B556-A31FA098E708%2FPilot%20Subscriptions%7C430EF17A-82B2-4182-862F-0F47E87AB1C6%2F%29) (MS FTE credentials required) 

In [ ]:
## specify subscription parameters
subscription_id='<<enter your subscription ID >>'    
workspace_name='<<a Unique Name -- use your alias>>'                                 

In [ ]:
from azureml.core.workspace import Workspace

location='eastus2'
resource_group='#<<enter your resource group>>'    

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = location,
                      exist_ok = True
                     )

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

This will take a few minutes, so let's talk about what a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) is while it is being created.
![](aml-workspace.png)


## Create a remote compute target
For this tutorial, we will create an AML Compute cluster with a NC6s_v3, v100 GPU machines, created to use as the [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to execute your training script on. 

**Creation of the cluster takes approximately 5 minutes, but we will not wait for it to complete** 

If the cluster is already in your workspace this code will skip the cluster creation process. Note that the code is not waiting for completion of the cluster creation. If needed you can call `compute_target.wait_for_completion(show_output=True)`, which will block you notebook until the compute target is provisioned.

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

# choose a name for your cluster
cluster_name = "nc6cluster"

try:
    compute_target = ws.compute_targets[cluster_name]
    print('Found existing compute target.')
except KeyError:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6_v3', 
                                                           idle_seconds_before_scaledown=1800,
                                                           min_nodes=0, 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

In [ ]:
# you can ask for the provisioning state to see whether the cluster is up already of if there were errors
compute_target.provisioning_state

## Attach the blobstore with the training data to the workspace
While the cluster is still creating, let's attach some data to our workspace.

The dataset we will use consists of ~150 images per class. Some breeds have more, while others have less. Each class has about 100 training images each for dog breeds, with ~50 validation images for each class. We will look at 10 classes in this tutorial.

To make the data accessible for remote training, you will need to keep the data in the cloud. AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data, and interact with it from your remote compute targets. It is an abstraction over Azure Storage. The datastore can reference either an Azure Blob container or Azure file share as the underlying storage. 

You can view the subset of the data used [here](https://github.com/heatherbshapiro/pycon-canada/tree/master/breeds-10). Or download it from [here](https://github.com/heatherbshapiro/pycon-canada/master/breeds-10.zip) as a zip file. 

In [ ]:
import os
import urllib
from zipfile import ZipFile

# download data
download_url = 'https://github.com/heatherbshapiro/pycon-canada/raw/master/breeds-10.zip'
data_file = './breeds-10.zip'
urllib.request.urlretrieve(download_url, filename=data_file)

# extract files
with ZipFile(data_file, 'r') as zip:
    print('extracting files...')
    zip.extractall()
    print('done')
    
# delete zip file
os.remove(data_file)

ds = ws.get_default_datastore()
ds.upload(src_dir='./breeds-10', target_path='breeds-10')

Now let's get a reference to the path on the datastore with the training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's `--data_dir` argument. We will start with the 10 classes dataset.

In [ ]:
ds = ws.get_default_datastore()
path_on_datastore = 'breeds-10'
ds_data = ds.path(path_on_datastore)
print(ds_data)

### Prepare training script
Now you will need to create your training script. In this tutorial, the training script is already provided for you at `pytorch_train.py`. In practice, you should be able to take any custom training script as is and run it with AML without having to modify your code.

However, if you would like to use AML's [tracking and metrics](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#metrics) capabilities, you will have to add a small amount of AML code inside your training script. 

In `pytorch_train.py`, we will log some metrics to our AML run. To do so, we will access the AML run object within the script:
```Python
from azureml.core.run import Run
run = Run.get_context()
```
Further within `pytorch_train.py`, we log the learning rate and momentum parameters, the best validation accuracy the model achieves, and the number of classes in the model:
```Python
run.log('lr', np.float(learning_rate))
run.log('momentum', np.float(momentum))
run.log('num_classes', num_classes)

run.log('best_val_acc', np.float(best_acc))
```

If you downloaded the data, you can start to train the model locally (note that it will take long if you don't have a GPU -- 21 min. on a Core i7 CPU).



## Train model on the remote compute
Now that you have your data and training script prepared, you are ready to train on your remote compute cluster. You can take advantage of Azure compute to leverage GPUs to cut down your training time. 

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this transfer learning PyTorch tutorial. 

In [ ]:
from azureml.core import Experiment

experiment_name = 'pytorch-dogs' 
experiment = Experiment(ws, name=experiment_name)

### Create a PyTorch estimator
The AML SDK's PyTorch estimator enables you to easily submit PyTorch training jobs for both single-node and distributed runs. For more information on the PyTorch estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-pytorch). The following code will define a single-node PyTorch job.

In [ ]:
##BATCH AI
from azureml.train.dnn import PyTorch

script_params = {
    '--data_dir': ds_data.as_mount(),
    '--num_epochs': 40,
    '--output_dir': './outputs',
    '--log_dir': './logs',
    '--mode': 'fine_tune'
}

estimator10 = PyTorch(source_directory='.', 
                    script_params=script_params,
                    compute_target=compute_target, 
                    entry_script='pytorch_train.py',
                    pip_packages=['tensorboardX'],
                    use_gpu=True)


The `script_params` parameter is a dictionary containing the command-line arguments to your training script `entry_script`. Please note the following:
- We passed our training data reference `ds_data` to our script's `--data_dir` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the training data `breeds` on our datastore.
- We specified the output directory as `./outputs`. The `outputs` directory is specially treated by AML in that all the content in this directory gets uploaded to your workspace as part of your run history. The files written to this directory are therefore accessible even once your remote run is over. In this tutorial, we will save our trained model to this output directory.

To leverage the Azure VM's GPU for training, we set `use_gpu=True`.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [ ]:
run10 = experiment.submit(estimator10)

### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run10).show()

### Hyperparameter Tuning
 Now that you have trained an initial model, you can tune the hyperparameters of this model to optimize model performance. Azure ML allows you to automate this tuning, in an efficient manner via early termination of poorly performing runs.

You can configure your Hyperparamter Tuning experiment by specifying the following info -
- Define the hyparparameter space - specify ranges, distribution and sampling
- Early Termination policy
- Optimization metric
- Resource / Compute budget
- Desired concurrency


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive import *

#define Random Sampling -
random_ps = RandomParameterSampling(
    {
        '--momentum': uniform(0.6,0.99),
        '--learning_rate': loguniform(-6, -3)
    }
)

#this is how you could use Grid Sampling instead -
grid_ps = GridParameterSampling( 
    {
        '--momentum': choice(0.6, 0.7, 0.8, 0.9),
        '--learning_rate': choice(0.02, 0.05, 0.1)
    }
)

#this is how you could use Bayesian Sampling -
bayesian_ps = BayesianParameterSampling( 
    {
        '--momentum': uniform(0.6,0.99),
        '--learning_rate': choice(0.02, 0.05, 0.1)
    }
)

#Bandit Early Termination Policy
bandit_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=10)

#this is how you could use Median Stopping Policy instead -
median_stopping_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

#this is how you could use Truncation Selection Policy -
truncation_selection_policy = TruncationSelectionPolicy(evaluation_interval=1, truncation_percentage=20, delay_evaluation=5)


hdc = HyperDriveRunConfig(estimator=estimator10, 
                          hyperparameter_sampling=random_ps, 
                          policy=bandit_policy, 
                          primary_metric_name='best_val_acc', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=50,
                          max_concurrent_runs=4)

hd_run = experiment.submit(hdc)
RunDetails(hd_run).show()

### Load a previously completed hyperparameter tuning experiment to visualize

In [ ]:
#use a previously completed hyperparameter tuning run
from azureml.core.experiment import Experiment
from azureml.train.hyperdrive import *
ps = RandomParameterSampling(
    {
        '--momentum': uniform(0.6,0.99),
        '--learning_rate': loguniform(-6, -3)
    }
)
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=10)
hdc = HyperDriveRunConfig(estimator=estimator10, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='best_val_acc', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=50,
                          max_concurrent_runs=4)
experiment = Experiment(ws, "pytorch-dogs")

hd_run_2=HyperDriveRun(experiment, "pytorch-dogs_1549158436181", hdc)

from azureml.widgets import RunDetails
RunDetails(hd_run_2).show(widget_settings={'debug':"true"})

### Find the config with the best performance
Once all of the hyperparameter tuning runs have completed, you can identify the best performing configuration and the corresponding hyperparameter value.

In [ ]:
best_run = hd_run_2.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['best_val_acc'])
print('\n momentum:',parameter_values[13])
print('\n learning rate:',parameter_values[11])

### Use the model from the best run for Inferencing

We can register the model from the best run and use it to deploy a web service that can be used for Inferencing. Details on how how you can do this can be found in the 'AI with Azure Machine Learning Service' webinar.

#### Thanks for following